# Scatering Network for Seismology

This notebook reproduces the tutorials from the SCATSEISNET package.

the package is ultra minimal. I needed the following installation commands to make it work
```bash
conda create -n scatnetseis python=3.8 pip
conda activate scatnetseis
pip install matplotlib
pip install scatnetseis
pip install jupyter
pip install obspy
python -m ipykernel install --user --name scatnetseis
```
After that you can start a jupyter notebook.

In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import obspy
from obspy.clients.fdsn.client import Client 
import sklearn
from sklearn.decomposition import FastICA

from scatseisnet import ScatteringNetwork
%config InlineBackend.figure_format = "svg"

ModuleNotFoundError: No module named 'sklearn'

### Scattering network parameters

Copy pasted from the github:

the number of octaves ( J, int) covered by the filter banks per layer. This defines the frequency range of analysis of the input data, from the Nyquist frequency fn down to fn/2^J , and should be decided according to the frequency range of interest for the task.

the resolution ( Q, int) represents the number of wavelets for each octave, so the frequency resolution of the filterbank. This should be large for the first layer (dense) and small for the other layers (sparse), as indicated in Andén and Mallat (2014).

the quality factor (float) is the ratio between the center frequency of every wavelet and the bandwidth. Because we work with constant-Q filters, this is defined from the entire filter bank. The lower the quality factor, the more redundant the information in the scattering coefficients. We suggest using a quality factor 1 at the first layer, and a larger at the remaining layers.

In [ ]:
segment_duration_seconds = 20.0
sampling_rate_hertz = 50.0
samples_per_segment = int(segment_duration_seconds * sampling_rate_hertz)
# the network will have 2 layers
bank_keyword_arguments = (
    {"octaves": 4, "resolution": 4, "quality": 1},
    {"octaves": 5, "resolution": 2, "quality": 3},
)

### Create scatnet

In [ ]:
network = ScatteringNetwork(
    *bank_keyword_arguments,
    bins=samples_per_segment,
    sampling_rate=sampling_rate_hertz,
)

print(network)

In [ ]:
dirpath_save = "./example"

# Create directory to save the results
os.makedirs(dirpath_save, exist_ok=True)

# Save the scattering network with Pickle
filepath_save = os.path.join(dirpath_save, "scattering_network.pickle")
with open(filepath_save, "wb") as file_save:
    pickle.dump(network, file_save, protocol=pickle.HIGHEST_PROTOCOL)

Visualize the filter bank

In [ ]:
for bank in network.banks:

    # Create axes (left for temporal, right for spectral domain)
    fig, ax = plt.subplots(1, 2, sharey=True)

    # Show each wavelet
    for wavelet, spectrum, ratio in zip(
        bank.wavelets, bank.spectra, bank.ratios
    ):

        # Time domain
        ax[0].plot(bank.times, wavelet.real + ratio, "C0")

        # Spectral domain (log of amplitude)
        ax[1].plot(bank.frequencies, np.log(np.abs(spectrum) + 1) + ratio, "C0")

    # Limit view to three times the temporal width of largest wavelet
    width_max = 3 * bank.widths.max()

    # Labels
    ax[0].set_ylabel("Octaves (base 2 log)")
    ax[0].set_xlabel("Time (seconds)")
    ax[0].set_xlim(-width_max, width_max)
    ax[0].grid()
    ax[1].set_xscale("log")
    ax[1].set_xlabel("Frequency (Hz)")
    ax[1].grid()


Load seismograms

In [ ]:

client = Client("IRIS")

# Collect waveforms from the datacenter
stream = client.get_waveforms(
    network="YH",
    station="DC08",
    location="*",
    channel="*",
    starttime=obspy.UTCDateTime("2012-07-25T00:00"),
    endtime=obspy.UTCDateTime("2012-07-26T00:00"),
)

stream.merge(method=1)
stream.detrend("linear")
stream.filter(type="highpass", freq=1.0)
stream.plot(rasterized=True);

stream.write("./example/scattering_stream.mseed", format="MSEED")

Trim Seismograms

In [ ]:
# Extract segment length (from any layer)
segment_duration = network.bins / network.sampling_rate
overlap = 0.5

# Gather list for timestamps and segments
timestamps = list()
segments = list()

# Collect data and timestamps
for traces in stream.slide(segment_duration, segment_duration * overlap):
    timestamps.append(mdates.num2date(traces[0].times(type="matplotlib")[0]))
    segments.append(np.array([trace.data[:-1] for trace in traces]))

Scattering transform

In [ ]:
scattering_coefficients = network.transform(segments, reduce_type=np.max)

In [ ]:
# Save the features in a pickle file
np.savez(
    "./example/scattering_coefficients.npz",
    order_1=scattering_coefficients[0],
    order_2=scattering_coefficients[1],
    times=timestamps,
)

observe the results

In [ ]:
# Extract the first channel
channel_id = 0
trace = stream[channel_id]
order_1 = np.log10(scattering_coefficients[0][:, channel_id, :].squeeze())
center_frequencies = network.banks[0].centers

# Create figure and axes
fig, ax = plt.subplots(2, sharex=True, dpi=300)

# Plot the waveform
ax[0].plot(trace.times("matplotlib"), trace.data, rasterized=True, lw=0.5)

# First-order scattering coefficients
ax[1].pcolormesh(timestamps, center_frequencies, order_1.T, rasterized=True)

# Axes labels
ax[1].set_yscale("log")
ax[0].set_ylabel("Counts")
ax[1].set_ylabel("Frequency (Hz)")

# Show
plt.show()

In [ ]:
# Reshape and stack scattering coefficients of all orders
order_1 = order_1.reshape(order_1.shape[0], -1)
order_2 = order_2.reshape(order_2.shape[0], -1)
scattering_coefficients = np.hstack((order_1, order_2))

# transform into log
scattering_coefficients = np.log(scattering_coefficients)

# print info about shape
n_times, n_coeff = scattering_coefficients.shape
print("Collected {} samples of {} dimensions each.".format(n_times, n_coeff))


## Dimensionality reduction

This tutorial uses FastICA for the dimensionality reduction, but we can try other things

In [ ]:
model = FastICA(n_components=10, whiten="unit-variance", random_state=42)
features = model.fit_transform(scattering_coefficients)


In [ ]:
# Save the features
np.savez(
    "./example/independent_components.npz",
    features=features,
    times=times,
)

# Save the dimension reduction model
with open("./example/dimension_model.pickle", "wb") as pickle_file:
    pickle.dump(
        model,
        pickle_file,
        protocol=pickle.HIGHEST_PROTOCOL,
    )

Plots the features

In [2]:
# Normalize features for display
features_normalized = features / np.abs(features).max(axis=0)

# Figure instance
fig = plt.figure(dpi=200)
ax = plt.axes()

# Plot features
ax.plot(times, features_normalized + np.arange(features.shape[1]), rasterized=True)

# Labels
ax.set_ylabel("Feature index")
ax.set_xlabel("Date and time")

# Show
plt.show()

NameError: name 'features' is not defined